In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

In [0]:
%sql

select * from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales/`

In [0]:
df_src = spark.sql("select distinct(Branch_ID),BranchName from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`")
df_src.display()

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    df_sink = spark.sql('select dim_branch_key,Branch_ID,BranchName from cars_catalog.gold.dim_branch')
else:
    df_sink = spark.sql('select 0 dim_branch_key,Branch_ID,BranchName from `delta`.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales` where 1=0')


In [0]:
df_sink.display()

In [0]:
df_filter = df_src.join(df_sink,df_src['Branch_ID'] == df_sink['Branch_ID'],'left').select(df_src['Branch_ID'],df_src['BranchName'],df_sink['dim_branch_key'])
df_filter.display()

In [0]:
df_filter_old = df_filter.filter(df_filter['dim_branch_key'].isNotNull())
df_filter_new = df_filter.filter(df_filter['dim_branch_key'].isNull()).select('Branch_ID','BranchName')
df_filter_old.display()
df_filter_new.display()

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
        max_value_df = spark.sql('select max(dim_branch_key) from  cars_catalog.gold.dim_branch')
        max_value = max_value_df.collect()[0][0]
if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    max_value_df = spark.sql('select max(dim_branch_key) from  cars_catalog.gold.dim_branch')
    max_value = max_value_df.collect()[0][0]
else:
    max_value = 1

In [0]:
df_filter_new = df_filter_new.withColumn('dim_branch_key',max_value+monotonically_increasing_id()+1)
df_filter_new.display()

In [0]:
df_final = df_filter_new.union(df_filter_old)
df_final.display()

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    delta_table = DeltaTable.forPath(spark,'abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_branch')
    delta_table.alias('t').merge(df_final.alias('s'),'t.dim_branch_key = s.dim_branch_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final.write.format('delta').mode('append').option('path','abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_branch').saveAsTable('cars_catalog.gold.dim_branch')
